<H3>PRI 2023/24: second
    project delivery</H3>

**GROUP 11**
- Francisco Martins, 99068
- Tunahan Güneş, 108108
- Sebastian Weidinger, 111612

In [ ]:
from transformers import BertTokenizer, BertModel
from G11_code.data_collection import *
from G11_code.helper_functions import *
from G11_code.indexing import *
from G11_code.evaluation import *
from G11_code.clustering import *

In [ ]:
article_path = os.path.join("..", "BBC News Summary", "BBC News Summary", "News Articles")
summary_path = os.path.join("..", "BBC News Summary", "BBC News Summary", "Summaries")
print("Article path:", article_path)
print("Summary path:", summary_path)
categorized_article_file_paths, categorized_articles, summary_file_paths, categorized_summaries, category_names= read_files(article_path, summary_path)

In [ ]:
categorized_summary_sentence_indices, faulty_summary_ids = get_summary_sentence_indices(categorized_articles, categorized_summaries)

In [ ]:
articles = remove_entries(categorized_articles, faulty_summary_ids)
article_file_paths = remove_entries(categorized_article_file_paths, faulty_summary_ids)
summaries = remove_entries(categorized_summaries, faulty_summary_ids)
summary_file_paths = remove_entries(summary_file_paths, faulty_summary_ids)

In [ ]:
I = indexing(articles)

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
bert_model = BertModel.from_pretrained("bert-base-uncased")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_model.to(device)

<H2>Main facilities</H2>

<h3>Part I: clustering</h3>

*A) Clustering*

In [ ]:
#code, statistics and/or charts here

*B) Summarization*

In [ ]:
#code, statistics and/or charts here

*C) Keyword extraction*

In [ ]:
#code, statistics and/or charts here

*D) Evaluation*

In [ ]:
#code, statistics and/or charts here

<h3>Part II: classification</h3>

*A) Feature extraction*

In [ ]:
#code and statistics here

*B) Classification*

In [ ]:
#code here

*C) Ranking extension*

In [ ]:
#code here

*D) Evaluation*

In [ ]:
#code, statistics and/or charts here

<H2>Question materials (optional)</H2>

<H3>Part I: clustering</H3>

**(1)** Do clustering-guided summarization alters the behavior and efficacy of the IR system?

In [ ]:
#code, statistics and/or charts here

**(2)** How sentence representations, clustering choices, and rank criteria impact summarization?

In [ ]:
#code, statistics and/or charts here

**...** (additional questions with empirical results)

<H3>END</H3>